## 필요한 라이브러리 불러오기

In [2]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# 한글깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [6]:
# Matplotlib에 나눔바른고딕 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

## 데이터 가공(공간정보)

In [24]:
# 병합할 기준 데이터 불러오기
data = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역/천안시 행정구역.csv')

In [25]:
data = data[['구','구분','이름']]

In [26]:
data

,구,구분,이름
0,동남구,읍·면,목천읍
1,동남구,읍·면,풍세면
2,동남구,읍·면,광덕면
3,동남구,읍·면,북면
4,동남구,읍·면,성남면
5,동남구,읍·면,수신면
6,동남구,읍·면,병천면
7,동남구,읍·면,동면
8,동남구,행정동명,중앙동
9,동남구,행정동명,문성동


In [27]:
from collections import defaultdict

# defaultdict를 사용하여 중복 key 문제 없이 다중 매핑 가능하게 함
legal_to_admin = defaultdict(list)

# 읍·면 (단일 매핑)
eup_myeon = [
    '목천읍', '풍세면', '광덕면', '북면',
    '성남면', '수신면', '병천면', '동면',
    '성환읍', '성거읍', '직산읍', '입장면'
]
for dong in eup_myeon:
    legal_to_admin[dong].append(dong)

# 동남구 행정동
legal_to_admin['대흥동'].append('중앙동')
legal_to_admin['오룡동'].append('중앙동')
legal_to_admin['사직동'].append('중앙동')
legal_to_admin['영성동'].append('중앙동')

legal_to_admin['문화동'].append('문성동')
legal_to_admin['성황동'].append('문성동')
legal_to_admin['원성동'].extend(['원성1동', '원성2동', '문성동'])  # 다중 매핑
legal_to_admin['유량동'].append('원성1동')

legal_to_admin['봉명동'].append('봉명동')

legal_to_admin['다가동'].append('일봉동')
legal_to_admin['용곡동'].extend(['일봉동', '신방동'])  # 중복 매핑
legal_to_admin['신방동'].append('신방동')

legal_to_admin['구성동'].append('청룡동')
legal_to_admin['청수동'].append('청룡동')
legal_to_admin['삼룡동'].append('청룡동')
legal_to_admin['청당동'].append('청룡동')
legal_to_admin['구룡동'].append('청룡동')

legal_to_admin['신부동'].append('신안동')
legal_to_admin['안서동'].append('신안동')

# 서북구 행정동
legal_to_admin['와촌동'].append('성정1동')
legal_to_admin['성정동'].extend(['성정1동', '성정2동'])

legal_to_admin['쌍용동'].extend(['쌍용1동', '쌍용2동', '쌍용3동'])

legal_to_admin['백석동'].append('백석동')

legal_to_admin['불당동'].extend(['불당1동', '불당2동'])

legal_to_admin['두정동'].append('부성2동')  # 구분 명확하게 하나로 설정
legal_to_admin['업성동'].append('부성1동')
legal_to_admin['신당동'].append('부성1동')
legal_to_admin['부대동'].append('부성1동')
legal_to_admin['성성동'].append('부성2동')
legal_to_admin['차암동'].append('부성2동')

# 리단위
legal_ri_map = {
    '목천읍': ['교천리', '교촌리', '남화리', '덕전리', '도장리', '동리', '동평리', '삼성리', '서리', '서흥리', '석천리', '소사리', '송전리', '신계리', '운전리', '응원리', '지산리', '천정리'],
    '풍세면': ['가송리', '남관리', '두남리', '미죽리', '보성리', '삼태리', '용정리', '풍서리'],
    '광덕면': ['광덕리', '대덕리', '대평리', '매당리', '무학리', '산원리', '신덕리', '신흥리', '원덕리', '지장리', '행정리'],
    '북면': ['납안리', '대평리', '매송리', '명덕리', '사담리', '상동리', '양곡리', '연춘리', '오곡리', '용암리', '운용리', '은지리', '전곡리'],
    '성남면': ['대정리', '대화리', '대흥리', '봉양리', '석곡리', '신덕리', '신사리', '용원리', '화성리'],
    '수신면': ['발산리', '백자리', '속창리', '신풍리', '장산리', '해정리'],
    '병천면': ['가전리', '관성리', '도원리', '매성리', '병천리', '봉항리', '송정리', '용두리', '탑원리'],
    '동면': ['광덕리', '구도리', '덕성리', '동산리', '송연리', '수남리', '장송리', '죽계리', '행암리', '화계리', '화덕리'],
    '성환읍': ['성환리', '성월리', '매주리', '율금리', '왕림리', '신방리', '송덕리', '우신리', '어룡리', '복모리', '신가리', '와룡리', '대홍리', '수향리', '안궁리', '양령리', '도하리', '학정리'],
    '성거읍': ['천흥리', '송남리', '오목리', '오색당리', '저리', '모전리', '삼곡리', '정촌리', '문덕리', '석교리', '요방리', '소우리', '신월리'],
    '직산읍': ['군동리', '군서리', '남산리', '판정리', '수헐리', '삼은리', '상덕리', '부송리', '모시리', '자은가리', '양당리', '신갈리', '마정리', '석곡리'],
    '입장면': ['하장리', '상장리', '기로리', '도림리', '양대리', '홍천리', '호당리', '시장리', '효계리', '신덕리', '신두리', '용정리', '흑암리', '산정리', '연곡리', '가산리', '유리', '독정리']
}

# 매핑 딕셔너리 생성
for admin_dong, ri_list in legal_ri_map.items():
    for legal in ri_list:
        legal_to_admin[legal].append(admin_dong)

# 완성된 딕셔너리 반환
dict(legal_to_admin)

{'목천읍': ['목천읍'],
 '풍세면': ['풍세면'],
 '광덕면': ['광덕면'],
 '북면': ['북면'],
 '성남면': ['성남면'],
 '수신면': ['수신면'],
 '병천면': ['병천면'],
 '동면': ['동면'],
 '성환읍': ['성환읍'],
 '성거읍': ['성거읍'],
 '직산읍': ['직산읍'],
 '입장면': ['입장면'],
 '대흥동': ['중앙동'],
 '오룡동': ['중앙동'],
 '사직동': ['중앙동'],
 '영성동': ['중앙동'],
 '문화동': ['문성동'],
 '성황동': ['문성동'],
 '원성동': ['원성1동', '원성2동', '문성동'],
 '유량동': ['원성1동'],
 '봉명동': ['봉명동'],
 '다가동': ['일봉동'],
 '용곡동': ['일봉동', '신방동'],
 '신방동': ['신방동'],
 '구성동': ['청룡동'],
 '청수동': ['청룡동'],
 '삼룡동': ['청룡동'],
 '청당동': ['청룡동'],
 '구룡동': ['청룡동'],
 '신부동': ['신안동'],
 '안서동': ['신안동'],
 '와촌동': ['성정1동'],
 '성정동': ['성정1동', '성정2동'],
 '쌍용동': ['쌍용1동', '쌍용2동', '쌍용3동'],
 '백석동': ['백석동'],
 '불당동': ['불당1동', '불당2동'],
 '두정동': ['부성2동'],
 '업성동': ['부성1동'],
 '신당동': ['부성1동'],
 '부대동': ['부성1동'],
 '성성동': ['부성2동'],
 '차암동': ['부성2동'],
 '교천리': ['목천읍'],
 '교촌리': ['목천읍'],
 '남화리': ['목천읍'],
 '덕전리': ['목천읍'],
 '도장리': ['목천읍'],
 '동리': ['목천읍'],
 '동평리': ['목천읍'],
 '삼성리': ['목천읍'],
 '서리': ['목천읍'],
 '서흥리': ['목천읍'],
 '석천리': ['목천읍'],
 '소사리': ['목천읍'],
 '송전리': ['목천읍'],
 '신계리

### **노인대중교통접근성취약지역**

#### 주소변환(2시간 소요)

In [ ]:
df1 = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역 가공 X/충청남도_공간정보 노인대중교통접근성취약지역.csv', encoding='cp949')

df1

,공간정보,공간아이디,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),여자(75-79세),남자(80-84세),여자(80-84세),남자(85-89세),여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상)
0,MULTIPOLYGON (((218694.0776824623 370229.22422...,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,MULTIPOLYGON (((210228.51662899548 362982.0548...,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,MULTIPOLYGON (((205857.32886523264 376264.4863...,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,MULTIPOLYGON (((215526.8521158972 363710.01422...,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,MULTIPOLYGON (((213042.10349218 360595.7824936...,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,MULTIPOLYGON (((214733.3422170509 362405.34174...,25796,2,0,0,2,2,0,1,2,2,1,0,2,2,0,0,2,2,0
12796,MULTIPOLYGON (((217330.66341908905 363120.7651...,25797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12797,MULTIPOLYGON (((209764.0133128531 375284.54926...,25799,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12798,MULTIPOLYGON (((211661.5799131109 375894.72786...,25800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
from shapely import wkt, ops
import pyproj
from tqdm import tqdm

# tqdm 적용
tqdm.pandas()

# 좌표계 설정 (EPSG:5181 → WGS84)
src_crs = pyproj.CRS("EPSG:5181")  # 또는 EPSG:5179
dst_crs = pyproj.CRS("EPSG:4326")
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True)

# 중심 위경도 추출 함수
def extract_center_latlon(wkt_str):
    try:
        geom = wkt.loads(wkt_str)
        geom_wgs84 = ops.transform(transformer.transform, geom)
        minx, miny, maxx, maxy = geom_wgs84.bounds
        center_lon = (minx + maxx) / 2
        center_lat = (miny + maxy) / 2
        return pd.Series({'lon': center_lon, 'lat': center_lat})
    except:
        return pd.Series({'lon': None, 'lat': None})

# 위경도 열 생성
df1[['lon', 'lat']] = df1['공간정보'].progress_apply(extract_center_latlon)

100%|██████████| 12800/12800 [00:11<00:00, 1094.55it/s]


In [ ]:
df1

,공간정보,공간아이디,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),여자(75-79세),...,남자(85-89세),여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상),lon,lat
0,MULTIPOLYGON (((218694.0776824623 370229.22422...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.210113,36.831008
1,MULTIPOLYGON (((210228.51662899548 362982.0548...,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.114704,36.765831
2,MULTIPOLYGON (((205857.32886523264 376264.4863...,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.065142,36.885555
3,MULTIPOLYGON (((215526.8521158972 363710.01422...,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.173356,36.772317
4,MULTIPOLYGON (((213042.10349218 360595.7824936...,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.145471,36.744292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,MULTIPOLYGON (((214733.3422170509 362405.34174...,25796,2,0,0,2,2,0,1,2,...,0,2,2,0,0,2,2,0,127.164442,36.760573
12796,MULTIPOLYGON (((217330.66341908905 363120.7651...,25797,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.194448,36.766942
12797,MULTIPOLYGON (((209764.0133128531 375284.54926...,25799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.108953,36.876693
12798,MULTIPOLYGON (((211661.5799131109 375894.72786...,25800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,127.130246,36.882170


In [ ]:
import requests

# Kakao REST API 키
KAKAO_API_KEY = 'KakaoAK 498d899fd084b23f98e07886d41630fc'

# 주소 캐시 딕셔너리
address_cache = {}

# 고유 좌표 추출
unique_coords = df1[['lon', 'lat']].drop_duplicates().dropna()

# Kakao 역지오코딩 함수
def kakao_reverse_geocode(lat, lon):
    try:
        url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        headers = {"Authorization": KAKAO_API_KEY}
        params = {"x": lon, "y": lat}
        response = requests.get(url, headers=headers, params=params, timeout=5)
        if response.status_code == 200:
            documents = response.json().get('documents')
            if documents:
                return documents[0]['address']['address_name']
    except:
        return None
    return None

# 고유 좌표에 대해 주소 조회
from tqdm import tqdm
for _, row in tqdm(unique_coords.iterrows(), total=len(unique_coords)):
    lat, lon = row['lat'], row['lon']
    key = (lat, lon)
    address_cache[key] = kakao_reverse_geocode(lat, lon)

# 주소 매핑
df1['실주소'] = df1.apply(lambda row: address_cache.get((row['lat'], row['lon'])), axis=1)

100%|██████████| 12800/12800 [2:10:21<00:00,  1.64it/s]


In [ ]:
df1

,공간정보,공간아이디,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),여자(75-79세),...,여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상),lon,lat,실주소
0,MULTIPOLYGON (((218694.0776824623 370229.22422...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.210113,36.831008,충남 천안시 동남구 목천읍 덕전리 산 35-1
1,MULTIPOLYGON (((210228.51662899548 362982.0548...,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.114704,36.765831,충남 아산시 배방읍 휴대리 산 46-1
2,MULTIPOLYGON (((205857.32886523264 376264.4863...,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.065142,36.885555,충남 아산시 음봉면 신휴리 산 66-1
3,MULTIPOLYGON (((215526.8521158972 363710.01422...,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.173356,36.772317,충남 천안시 동남구 목천읍 응원리 267
4,MULTIPOLYGON (((213042.10349218 360595.7824936...,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.145471,36.744292,충남 천안시 동남구 풍세면 미죽리 산 11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,MULTIPOLYGON (((214733.3422170509 362405.34174...,25796,2,0,0,2,2,0,1,2,...,2,2,0,0,2,2,0,127.164442,36.760573,충남 천안시 동남구 목천읍 삼성리 산 48
12796,MULTIPOLYGON (((217330.66341908905 363120.7651...,25797,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.194448,36.766942,충남 천안시 동남구 목천읍 응원리 산 28-3
12797,MULTIPOLYGON (((209764.0133128531 375284.54926...,25799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.108953,36.876693,충남 천안시 서북구 직산읍 양당리 산 40-1
12798,MULTIPOLYGON (((211661.5799131109 375894.72786...,25800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.130246,36.882170,충남 천안시 서북구 직산읍 부송리 산 4


In [ ]:
df1.to_csv('노인대중교통접근성취약지역_주소변환.csv', index=False, encoding='cp949')

#### 저장된 데이터 불러와서 가공 진행

In [76]:
df1 = pd.read_csv('/content/노인대중교통접근성취약지역_주소변환.csv', encoding='cp949')

df1

,공간정보,공간아이디,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),여자(75-79세),...,여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상),lon,lat,실주소
0,MULTIPOLYGON (((218694.0776824623 370229.22422...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.210113,36.831008,충남 천안시 동남구 목천읍 덕전리 산 35-1
1,MULTIPOLYGON (((210228.51662899548 362982.0548...,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.114704,36.765831,충남 아산시 배방읍 휴대리 산 46-1
2,MULTIPOLYGON (((205857.32886523264 376264.4863...,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.065142,36.885555,충남 아산시 음봉면 신휴리 산 66-1
3,MULTIPOLYGON (((215526.8521158972 363710.01422...,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.173356,36.772317,충남 천안시 동남구 목천읍 응원리 267
4,MULTIPOLYGON (((213042.10349218 360595.7824936...,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.145471,36.744292,충남 천안시 동남구 풍세면 미죽리 산 11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,MULTIPOLYGON (((214733.3422170509 362405.34174...,25796,2,0,0,2,2,0,1,2,...,2,2,0,0,2,2,0,127.164442,36.760573,충남 천안시 동남구 목천읍 삼성리 산 48
12796,MULTIPOLYGON (((217330.66341908905 363120.7651...,25797,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.194448,36.766942,충남 천안시 동남구 목천읍 응원리 산 28-3
12797,MULTIPOLYGON (((209764.0133128531 375284.54926...,25799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.108953,36.876693,충남 천안시 서북구 직산읍 양당리 산 40-1
12798,MULTIPOLYGON (((211661.5799131109 375894.72786...,25800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.130246,36.882170,충남 천안시 서북구 직산읍 부송리 산 4


In [77]:
df1 = df1.dropna(axis=0)

In [78]:
# 천인시만 필터링
df1 = df1[df1['실주소'].str.contains('천안시')]

df1

,공간정보,공간아이디,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),여자(75-79세),...,여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상),lon,lat,실주소
0,MULTIPOLYGON (((218694.0776824623 370229.22422...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.210113,36.831008,충남 천안시 동남구 목천읍 덕전리 산 35-1
3,MULTIPOLYGON (((215526.8521158972 363710.01422...,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.173356,36.772317,충남 천안시 동남구 목천읍 응원리 267
4,MULTIPOLYGON (((213042.10349218 360595.7824936...,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.145471,36.744292,충남 천안시 동남구 풍세면 미죽리 산 11-16
6,MULTIPOLYGON (((216603.21802933697 368317.5119...,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.186635,36.813675,충남 천안시 동남구 유량동 60-1
7,MULTIPOLYGON (((218806.14954145873 367928.8675...,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.210191,36.810275,충남 천안시 동남구 목천읍 덕전리 산 79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,MULTIPOLYGON (((214733.3422170509 362405.34174...,25796,2,0,0,2,2,0,1,2,...,2,2,0,0,2,2,0,127.164442,36.760573,충남 천안시 동남구 목천읍 삼성리 산 48
12796,MULTIPOLYGON (((217330.66341908905 363120.7651...,25797,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.194448,36.766942,충남 천안시 동남구 목천읍 응원리 산 28-3
12797,MULTIPOLYGON (((209764.0133128531 375284.54926...,25799,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.108953,36.876693,충남 천안시 서북구 직산읍 양당리 산 40-1
12798,MULTIPOLYGON (((211661.5799131109 375894.72786...,25800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,127.130246,36.882170,충남 천안시 서북구 직산읍 부송리 산 4


In [79]:
from collections import defaultdict
import pandas as pd

# 1. df1의 인구 관련 컬럼 추출
population_columns = [col for col in df1.columns if '남자' in col or '여자' in col]

# 2. data에 대응하는 값을 저장할 빈 데이터프레임 생성
merged3 = pd.DataFrame(0, index=range(len(data)), columns=population_columns)

# 3. 이름 → 인덱스 매핑 딕셔너리
name_to_index = {name: idx for idx, name in enumerate(data['이름'])}

# 4. df1['실주소']에서 legal_to_admin의 key가 포함된 경우, 그 key에 해당하는 모든 행정동에 대해 값을 누적
for i, row in df1.iterrows():
    address = row['실주소']
    for legal, admins in legal_to_admin.items():
        if legal in address:
            for admin in admins:
                if admin in name_to_index:
                    idx = name_to_index[admin]
                    merged3.loc[idx] += row[population_columns]
            break  # 하나의 주소가 여러 legal 명칭에 매칭되는 경우 방지 (선택적)

# 5. merged3와 data 병합
merged1 = pd.concat([data.reset_index(drop=True), merged3], axis=1)

In [80]:
merged1

,구,구분,이름,남자(60-64세),여자(60-64세),남자(65-69세),여자(65-69세),남자(70-74세),여자(70-74세),남자(75-79세),...,남자(80-84세),여자(80-84세),남자(85-89세),여자(85-89세),남자(90-94세),여자(90-94세),남자(95-99세),여자(95-99세),남자(100세이상),여자(100세이상)
0,동남구,읍·면,목천읍,175,0,1,175,175,1,7,...,175,11,10,175,175,20,20,175,175,36
1,동남구,읍·면,풍세면,321,1,6,321,321,11,13,...,321,16,33,321,321,40,39,321,321,63
2,동남구,읍·면,광덕면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,동남구,읍·면,북면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,동남구,읍·면,성남면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,동남구,읍·면,수신면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,동남구,읍·면,병천면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,동남구,읍·면,동면,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,동남구,행정동명,중앙동,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,동남구,행정동명,문성동,6,0,0,6,6,0,0,...,6,0,0,6,6,1,2,6,6,2


In [81]:
from collections import defaultdict

name_to_indices = defaultdict(set)  # 각 이름별 포함된 행 인덱스
unique_matched_rows = set()         # 전체에서 중복 제거된 행 인덱스

for i, row in df1.iterrows():
    address = row['실주소']
    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_to_indices[admin].add(i)
                    unique_matched_rows.add(i)  # 모든 이름에서 중복 없이 포함된 행 모음
            break  # 첫 번째 일치한 법정동만 사용하여 중복 방지

# 이름별 포함된 행 수 출력
total = 0
for name in data['이름']:
    count = len(name_to_indices[name])
    print(f"🔹 {name}: {count}개 행 포함")
    total += count

# 최종 통계
print(f"\n이름별 포함된 행 수 합계 (중복 포함): {total}")
print(f"중복 없이 포함된 전체 행 수: {len(unique_matched_rows)}")
print(f"df1 전체 행 수: {len(df1)}")

🔹 목천읍: 1194개 행 포함
🔹 풍세면: 968개 행 포함
🔹 광덕면: 0개 행 포함
🔹 북면: 0개 행 포함
🔹 성남면: 0개 행 포함
🔹 수신면: 0개 행 포함
🔹 병천면: 0개 행 포함
🔹 동면: 0개 행 포함
🔹 중앙동: 0개 행 포함
🔹 문성동: 19개 행 포함
🔹 원성1동: 451개 행 포함
🔹 원성2동: 19개 행 포함
🔹 봉명동: 5개 행 포함
🔹 일봉동: 36개 행 포함
🔹 신방동: 151개 행 포함
🔹 청룡동: 817개 행 포함
🔹 신안동: 533개 행 포함
🔹 성환읍: 150개 행 포함
🔹 성거읍: 502개 행 포함
🔹 직산읍: 1032개 행 포함
🔹 입장면: 0개 행 포함
🔹 성정1동: 23개 행 포함
🔹 성정2동: 23개 행 포함
🔹 쌍용1동: 70개 행 포함
🔹 쌍용2동: 70개 행 포함
🔹 쌍용3동: 70개 행 포함
🔹 백석동: 26개 행 포함
🔹 불당1동: 45개 행 포함
🔹 불당2동: 45개 행 포함
🔹 부성1동: 514개 행 포함
🔹 부성2동: 146개 행 포함

이름별 포함된 행 수 합계 (중복 포함): 6909
중복 없이 포함된 전체 행 수: 6627
df1 전체 행 수: 6627


In [82]:
# 가공된 파일 csv 다운로드
from google.colab import files
merged1.to_csv('천안시_노인대중교통접근성취약지역.csv', index=False, encoding='utf-8-sig')
files.download('천안시_노인대중교통접근성취약지역.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**가공 과정**

1. polygon 정보 > 위경도로 변환
2. 위경도 좌표 > 카카오 API 사용하여 실주소로 변경 (2시간 걸림...)
3. 실주소 칼럼 생성 후 csv 저장
----
<저장된 csv 파일 불러와서>
4. NaN 존재하는 행 제거
5. 실주소에 '천안시' 해당되는 행만 추출
6. legal_to_admin 함수 이용하여 같은 행정동에 포함되는 행 찾은 후 각 행 값 합산하여 dataframe 생성
7. 행정구역 csv에 해당 dataframe 병합하여 새로운 csv 파일 생성

### **노인돌봄기본서비스수행기관**

In [71]:
df2 = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역 가공 X/충청남도_공간정보 노인돌봄기본서비스수행기관.csv', encoding='cp949')

df2

,공간정보,시군구,기관명,주소
0,POINT (223280.626 308312.544),계룡시,계룡시노인복지관,충청남도 계룡시 금암동 56
1,POINT (212489.225 331120.029),공주시,공주노인복지센터,충청남도 공주시 금흥동 616-6
2,POINT (243939.535 290234.259),금산군,금산군청,충청남도 금산군 금산읍 상리 25-1
3,POINT (210588.587 298589.151),논산시,논산시청 독거노인돌봄센터,충청남도 논산시 관촉동 19-1
4,POINT (168226.13 377044.722),당진시,당진시노인복지관,충청남도 당진시 수청동 1005
5,POINT (164473.954 317646.293),보령시,보령노인종합복지관,충청남도 보령시 죽정동 703-1
6,POINT (191891.645 308607.658),부여군,부여군청 독거노인유케어센터,충청남도 부여군 부여읍 동남리 725
7,POINT (151943.286 363894.019),서산시,서산석림사회복지관,충청남도 서산시 석림동 463-3
8,POINT (169110.987 290419.63),서천군,서천군노인복지관,충청남도 서천군 종천면 종천리 37-3
9,POINT (199575.684 364826.15),아산시,아산시노인종합복지관,충청남도 아산시 온천동 266-35


In [72]:
# 천인시만 필터링
df2 = df2[df2['시군구'] == '천안시']

df2

,공간정보,시군구,기관명,주소
11,POINT (211528.506 367295.576),천안시,천안시노인종합복지관,충청남도 천안시 서북구 쌍용동 1038


### **노인복지시설**

In [83]:
df3 = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역 가공 X/충청남도_공간정보 노인복지시설.csv', encoding='cp949')

df3

,공간정보,구분,시군,기능,이름,주소
0,POINT (220601.366 362997.572),노인주거복지시설,천안시,양로시설,천안실버타운,천안시 동남구 목천읍 운전4길 6
1,POINT (220674.372 362764.577),노인주거복지시설,천안시,양로시설,뉴천안실버타운,천안시 동남구 목천읍 운전4길 20
2,POINT (215737.426 363151.422),노인주거복지시설,천안시,노인공동생활가정,동산실버타운,천안시 동남구 목천읍 응원1길 69-35
3,POINT (215737.426 363151.422),노인의료복지시설,천안시,노인요양시설(개정법),동산요양원,천안시 목천읍 응원1길 69-35
4,POINT (221058.346 363434.58),노인의료복지시설,천안시,노인요양시설(개정법),목천요양원,천안시 목천읍 충절로 1156
...,...,...,...,...,...,...
406,POINT (130592.672 360481.907),노인의료복지시설,태안군,노인요양공동생활가정,새롬요양원,태안군 근흥면 낭금길 455-4(마금리)
407,POINT (124053.637 364756.653),노인의료복지시설,태안군,노인요양시설(개정법),벧엘요양원,태안군 소원면 서해로 119-30(모항리)
408,POINT (135647.085 377865.812),노인의료복지시설,태안군,노인요양시설(개정법),사회복지법인 서혜원,태안군 이원면 발전로 939(관리)
409,POINT (135647.085 377865.812),재가노인복지시설,태안군,"단기보호, 주야간보호",사회복지법인 서혜원,태안군 이원면 발전로 939(관리)


In [84]:
# 천인시만 필터링
df3 = df3[df3['시군'] == '천안시']

df3

,공간정보,구분,시군,기능,이름,주소
0,POINT (220601.366 362997.572),노인주거복지시설,천안시,양로시설,천안실버타운,천안시 동남구 목천읍 운전4길 6
1,POINT (220674.372 362764.577),노인주거복지시설,천안시,양로시설,뉴천안실버타운,천안시 동남구 목천읍 운전4길 20
2,POINT (215737.426 363151.422),노인주거복지시설,천안시,노인공동생활가정,동산실버타운,천안시 동남구 목천읍 응원1길 69-35
3,POINT (215737.426 363151.422),노인의료복지시설,천안시,노인요양시설(개정법),동산요양원,천안시 목천읍 응원1길 69-35
4,POINT (221058.346 363434.58),노인의료복지시설,천안시,노인요양시설(개정법),목천요양원,천안시 목천읍 충절로 1156
...,...,...,...,...,...,...
76,POINT (212433.257 370207.227),노인의료복지시설,천안시,노인요양시설(개정법),효누리요양원,"천안시 서북구 오성로 107, 8층(두정동)"
77,POINT (211395.287 369674.202),노인의료복지시설,천안시,노인요양시설(개정법),늘푸른요양원1,천안시 서북구 늘푸른3길 7-1 201호(두정동)
78,POINT (211395.287 369674.202),노인의료복지시설,천안시,노인요양시설(개정법),늘푸른요양원2,천안시 서북구 늘푸른3길 7-1 301(성정동)
79,POINT (211395.287 369674.202),노인의료복지시설,천안시,노인요양시설(개정법),성모효드림요양원,"천안시 서북구 늘푸른3길 7-1, 401호(두정동)"


In [113]:
df3['기능'].unique()

array(['양로시설', '노인공동생활가정', '노인요양시설(개정법)', '단기보호, 방문요양, 주야간보호',
       '방문목욕, 방문요양', '노인복지관', '주야간보호, 방문요양, 재가노인지원서비스', '방문요양',
       '노인요양공동생활가정', '주야간보호', '방문목욕, 방문요양, 재가노인지원서비스',
       '방문요양, 주야간보호, 재가노인지원서비스', '재가노인지원서비스',
       '방문목욕, 방문요양, 주야간보호, 재가노인지원서비스'], dtype=object)

In [107]:
import pandas as pd
from collections import defaultdict

# 1. 이름 → 인덱스 매핑
name_to_index = {name: idx for idx, name in enumerate(data['이름'])}

# 2. 이름별 시설 기능 개수 초기화
name_func_counts = defaultdict(lambda: defaultdict(int))

# 3. 주소 기준으로 legal_to_admin 통해 이름 매칭 후 기능 카운트
for i, row in df3.iterrows():
    address = row['주소']
    func = row['기능']  # 예: '노인요양시설', '노인요양공동생활가정' 등

    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_func_counts[admin][func] += 1
            break  # 중복 방지: 하나의 주소에 여러 legal이 걸리지 않도록

# 4. 데이터프레임으로 변환
merged3 = data.copy()

# 5. 고유한 시설 기능 파악
all_funcs = set()
for func_counts in name_func_counts.values():
    all_funcs.update(func_counts.keys())

# 6. 각 시설 기능별 컬럼 채우기
for func in sorted(all_funcs):
    merged3[func] = merged3['이름'].apply(lambda x: name_func_counts[x][func])

# 결측값 0으로 채우기
merged3.fillna(0, inplace=True)

# 정수형 변환
for func in sorted(all_funcs):
    merged3[func] = merged3[func].astype(int)

In [116]:
merged3

,구,구분,이름,노인공동생활가정,노인복지관,노인요양공동생활가정,노인요양시설(개정법),"단기보호, 방문요양, 주야간보호","방문목욕, 방문요양","방문목욕, 방문요양, 재가노인지원서비스","방문목욕, 방문요양, 주야간보호, 재가노인지원서비스",방문요양,"방문요양, 주야간보호, 재가노인지원서비스",양로시설,재가노인지원서비스,주야간보호,"주야간보호, 방문요양, 재가노인지원서비스"
0,동남구,읍·면,목천읍,1,0,0,5,0,0,0,0,0,0,2,0,0,0
1,동남구,읍·면,풍세면,0,0,0,2,0,0,0,0,0,0,0,0,0,0
2,동남구,읍·면,광덕면,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,동남구,읍·면,북면,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,동남구,읍·면,성남면,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,동남구,읍·면,수신면,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,동남구,읍·면,병천면,0,1,0,1,0,0,0,0,1,0,0,0,0,1
7,동남구,읍·면,동면,1,0,0,1,0,0,0,0,0,0,1,0,0,0
8,동남구,행정동명,중앙동,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,동남구,행정동명,문성동,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [129]:
from collections import defaultdict

name_to_index = {name: idx for idx, name in enumerate(data['이름'])}
name_to_indices = defaultdict(set)
unique_matched_rows = set()

for i, row in df3.iterrows():
    address = row['주소']
    matched = False
    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_to_indices[admin].add(i)
                    unique_matched_rows.add(i)
                    matched = True
            break  # 첫 번째 일치 legal만 사용
    # 매칭이 안 된 경우에 대한 별도 처리도 여기에 가능

# 이름별 포함된 행 수 출력
total = 0
print("🔹 이름별 포함된 df3 행 수:")
for name in data['이름']:
    count = len(name_to_indices[name])
    print(f"  {name}: {count}개")
    total += count

print(f"\n이름별 포함된 df3 행 수 합계 (중복 포함): {total}")
print(f"중복 없이 포함된 전체 df3 행 수: {len(unique_matched_rows)}")
print(f"df3 전체 행 수: {len(df3)}")

🔹 이름별 포함된 df3 행 수:
  목천읍: 8개
  풍세면: 2개
  광덕면: 2개
  북면: 1개
  성남면: 1개
  수신면: 0개
  병천면: 4개
  동면: 3개
  중앙동: 0개
  문성동: 1개
  원성1동: 2개
  원성2동: 1개
  봉명동: 3개
  일봉동: 4개
  신방동: 6개
  청룡동: 6개
  신안동: 1개
  성환읍: 8개
  성거읍: 3개
  직산읍: 4개
  입장면: 2개
  성정1동: 3개
  성정2동: 3개
  쌍용1동: 3개
  쌍용2동: 3개
  쌍용3동: 3개
  백석동: 6개
  불당1동: 2개
  불당2동: 2개
  부성1동: 1개
  부성2동: 8개

이름별 포함된 df3 행 수 합계 (중복 포함): 96
중복 없이 포함된 전체 df3 행 수: 79
df3 전체 행 수: 81


In [130]:
# ✅ 이름에 포함되지 않은 행들 정확하게 출력
unmatched_df3 = df3[~df3.index.isin(unique_matched_rows)]
print(f"이름에 포함되지 않은 df4 행 수: {len(unmatched_df3)}")
print(unmatched_df3)

이름에 포함되지 않은 df4 행 수: 2
                             공간정보        구분   시군          기능          이름  \
41  POINT (211493.981 379712.069)  노인의료복지시설  천안시  노인요양공동생활가정     비타민2요양원   
59  POINT (211517.357 367306.238)  노인여가복지시설  천안시       노인복지관  천안시노인종합복지관   

                       주소   행정동  
41  천안시 서북구 성환9길9, 광성빌딩2층  None  
59       천안시 서북구 미라11길 16  None  


In [118]:
# 가공된 파일 csv 다운로드
from google.colab import files
merged3.to_csv('천안시_노인복지시설.csv', index=False, encoding='utf-8-sig')
files.download('천안시_노인복지시설.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**가공 과정**

천안시 필터링 후 기존 데이터 행정동 매핑 진행하고 시설별로 숫자 카운팅하여 csv 생성

### **노인시설**

In [119]:
df4 = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역 가공 X/충청남도_공간정보 노인시설.csv', encoding='cp949')

df4

,시설구분,시설명,주소,연락처
0,노인요양시설,충무원,천안시 동남구 목천읍 삼방로 731,041-558-7590
1,경로식당,(사)대한노인회천안시지회,천안시 동남구 중앙로 217(신부동 370-6),041-565-0247
2,경로식당,(사)상록회천안지회,천안시 동남구 청수동 266-6,041-562-9294
3,경로식당,(사)충남카톨릭사회복지회 천안 성모의집,천안시 동남구 원성9길 3(원성2동 491-6),041-552-2046
4,경로식당,(사)흙과샘,청양군 화성면 구봉로17번지(산정리231-1),041-943-1669
...,...,...,...,...
665,노인요양시설,효자의 집,천안시 동남구 충절로 535-18 (삼룡동),041-558-7775
666,노인요양시설,효제요양원,당진시 고대면 온동1길 120-20,041-353-8877
667,노인요양공동생활가정,효진원,금산군 금산읍 계진길 58,041-754-7976
668,노인요양시설,휴드림요양원,부여군 외산면 만수로 133,041-836-2223


In [120]:
# 천인시만 필터링
df4 = df4[df4['주소'].str.contains('천안시')]

df4

,시설구분,시설명,주소,연락처
0,노인요양시설,충무원,천안시 동남구 목천읍 삼방로 731,041-558-7590
1,경로식당,(사)대한노인회천안시지회,천안시 동남구 중앙로 217(신부동 370-6),041-565-0247
2,경로식당,(사)상록회천안지회,천안시 동남구 청수동 266-6,041-562-9294
3,경로식당,(사)충남카톨릭사회복지회 천안 성모의집,천안시 동남구 원성9길 3(원성2동 491-6),041-552-2046
11,노인요양시설,가강요양원,"천안시 서북구 두정중11길 14, 401호(두정동)",041-567-0133
...,...,...,...,...
630,노인요양시설,행복나눔노인요양원,천안시 동남구 태조산길 266 (유량동),041-523-7555
635,노인요양공동생활가정,행복미소의집,"천안시 서북구 백석로 136, 113동 301호",041-555-8755
644,노인요양시설,호서노인전문요양원,천안시 동남구 충절로 535-18 (삼룡동),041-558-7772
654,노인요양시설,효누리요양원,천안시 서북구 오성로 107. 8층,041-555-5570


In [121]:
df4['시설구분'].unique()

array(['노인요양시설', '경로식당', '노인요양공동생활가정', '재가노인복지시설 단기보호', '재가노인복지시설 방문목욕',
       '재가노인복지시설 방문요양', '재가노인복지시설 주야간보호', '단기보호', '방문요양', '주야간보호',
       '재가노인복지시설 재가노인지원서비스', '재가노인지원서비스', '양로시설', '공동생활가정', '방문목욕',
       '노인복지관'], dtype=object)

In [122]:
import pandas as pd
from collections import defaultdict

# 1. 이름 → 인덱스 매핑
name_to_index = {name: idx for idx, name in enumerate(data['이름'])}

# 2. 이름별 시설 기능 개수 초기화
name_func_counts = defaultdict(lambda: defaultdict(int))

# 3. 주소 기준으로 legal_to_admin 통해 이름 매칭 후 기능 카운트
for i, row in df4.iterrows():
    address = row['주소']
    func = row['시설구분']

    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_func_counts[admin][func] += 1
            break  # 중복 방지: 하나의 주소에 여러 legal이 걸리지 않도록

# 4. 데이터프레임으로 변환
merged4 = data.copy()

# 5. 고유한 시설 기능 파악
all_funcs = set()
for func_counts in name_func_counts.values():
    all_funcs.update(func_counts.keys())

# 6. 각 시설 기능별 컬럼 채우기
for func in sorted(all_funcs):
    merged4[func] = merged4['이름'].apply(lambda x: name_func_counts[x][func])

# 결측값 0으로 채우기
merged4.fillna(0, inplace=True)

# 정수형 변환
for func in sorted(all_funcs):
    merged4[func] = merged4[func].astype(int)

In [123]:
merged4

,구,구분,이름,경로식당,공동생활가정,노인요양공동생활가정,노인요양시설,단기보호,방문목욕,방문요양,양로시설,재가노인복지시설 단기보호,재가노인복지시설 방문목욕,재가노인복지시설 방문요양,재가노인복지시설 재가노인지원서비스,재가노인복지시설 주야간보호,재가노인지원서비스,주야간보호
0,동남구,읍·면,목천읍,0,1,0,5,0,0,1,2,0,0,0,0,0,0,1
1,동남구,읍·면,풍세면,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0
2,동남구,읍·면,광덕면,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1
3,동남구,읍·면,북면,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,동남구,읍·면,성남면,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,동남구,읍·면,수신면,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,동남구,읍·면,병천면,1,0,0,1,0,0,2,0,0,1,2,1,1,1,1
7,동남구,읍·면,동면,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
8,동남구,행정동명,중앙동,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,동남구,행정동명,문성동,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [131]:
import pandas as pd
from collections import defaultdict

name_to_index = {name: idx for idx, name in enumerate(data['이름'])}
name_to_indices = defaultdict(set)
unique_matched_rows = set()

for i, row in df4.iterrows():
    address = row['주소']
    matched = False
    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_to_indices[admin].add(i)
                    unique_matched_rows.add(i)
                    matched = True
            break  # 첫 번째 일치 legal만 사용
    # 여기에서 매칭이 안 된 경우만 기록
    if not matched:
        row['index'] = i
        # 나중에 매칭 안 된 행 수 확인용
        pass

# 이름별 포함된 행 수 출력
total = 0
print("🔹 이름별 포함된 df4 행 수:")
for name in data['이름']:
    count = len(name_to_indices[name])
    print(f"  {name}: {count}개")
    total += count

print(f"\n이름별 포함된 df4 행 수 합계 (중복 포함): {total}")
print(f"중복 없이 포함된 전체 df4 행 수: {len(unique_matched_rows)}")
print(f"df4 전체 행 수: {len(df4)}")

🔹 이름별 포함된 df4 행 수:
  목천읍: 10개
  풍세면: 4개
  광덕면: 7개
  북면: 1개
  성남면: 4개
  수신면: 0개
  병천면: 11개
  동면: 2개
  중앙동: 1개
  문성동: 1개
  원성1동: 2개
  원성2동: 1개
  봉명동: 4개
  일봉동: 4개
  신방동: 11개
  청룡동: 7개
  신안동: 5개
  성환읍: 14개
  성거읍: 3개
  직산읍: 5개
  입장면: 6개
  성정1동: 2개
  성정2동: 2개
  쌍용1동: 7개
  쌍용2동: 7개
  쌍용3동: 7개
  백석동: 3개
  불당1동: 5개
  불당2동: 5개
  부성1동: 1개
  부성2동: 9개

이름별 포함된 df4 행 수 합계 (중복 포함): 151
중복 없이 포함된 전체 df4 행 수: 124
df4 전체 행 수: 142


In [132]:
# ✅ 이름에 포함되지 않은 행들 정확하게 출력
unmatched_df4 = df4[~df4.index.isin(unique_matched_rows)]
print(f"이름에 포함되지 않은 df4 행 수: {len(unmatched_df4)}")
print(unmatched_df4)

이름에 포함되지 않은 df4 행 수: 18
           시설구분                    시설명                                주소  \
3          경로식당  (사)충남카톨릭사회복지회 천안 성모의집        천안시 동남구 원성9길 3(원성2동 491-6)   
88       노인요양시설             김태희치매전문요양원  천안시 서북구 두정중 11길 14 한길프라자 B동 201호   
195  노인요양공동생활가정                   미소의집        천안시 서북구 백석로 136, 113동 603호   
200      노인요양시설                  백석요양원       천안시 서북구 백석4길 12 거성캐슬빌딩 701호   
226      노인요양시설                 봉서산요양원                천안시 서북구 미라16길 38-7   
398       노인복지관               아우내은빛복지관                   천안시 동남구 병천2로 51   
490      노인요양시설               이화엔젤스요양원                 천안시 동남구 대흥로 336-4   
526  노인요양공동생활가정                   참조은집               천안시 서북구 동서대로 110-23   
532       주야간보호         천안고향의집병설주간보호센터                   천안시 동남구 우영3길 21   
537       노인복지관             천안시노인종합복지관                 천안시 서북구 미라 11길 16   
542   재가노인지원서비스  천안시노인종합복지관부설 천안노인복지센터                  천안시 서북구 미라11길 16   
543        방문목욕  천안시노인종합복지관부설 천안노인복지센터                  천안시 서북구 

In [133]:
# 가공된 파일 csv 다운로드
from google.colab import files
merged4.to_csv('천안시_노인시설.csv', index=False, encoding='utf-8-sig')
files.download('천안시_노인시설.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**가공 과정**

천안시 필터링 후 기존 데이터 행정동 매핑 진행하고 시설별로 숫자 카운팅하여 csv 생성

### **노인인구**

#### 주소변환(2시간 소요)

In [7]:
df5 = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역 가공 X/충청남도_공간정보 노인인구.csv', encoding='cp949')

df5

,공간정보,나이,성별
0,POINT (211863.24 369313.275),80세 ~ 84세,F
1,POINT (211863.24 369313.275),70세 ~ 74세,F
2,POINT (138316.292 362843.505),85세 ~ 89세,F
3,POINT (166462.996 378076.684),85세 ~ 89세,F
4,POINT (166462.996 378076.684),90세 ~ 94세,F
...,...,...,...
96168,POINT (202829.446 318741.55),70세 ~ 74세,M
96169,POINT (208213.031 320403.21),75세 ~ 79세,F
96170,POINT (208186.245 320397.197),80세 ~ 84세,M
96171,POINT (208186.245 320397.197),75세 ~ 79세,F


In [8]:
import pandas as pd
from shapely import wkt, ops
import pyproj
from tqdm import tqdm

# tqdm 적용
tqdm.pandas()

# 좌표계 설정 (EPSG:5181 → WGS84)
src_crs = pyproj.CRS("EPSG:5181")  # 또는 EPSG:5179
dst_crs = pyproj.CRS("EPSG:4326")
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True)

# 중심 위경도 추출 함수
def extract_center_latlon(wkt_str):
    try:
        geom = wkt.loads(wkt_str)
        geom_wgs84 = ops.transform(transformer.transform, geom)
        minx, miny, maxx, maxy = geom_wgs84.bounds
        center_lon = (minx + maxx) / 2
        center_lat = (miny + maxy) / 2
        return pd.Series({'lon': center_lon, 'lat': center_lat})
    except:
        return pd.Series({'lon': None, 'lat': None})

# 위경도 열 생성
df5[['lon', 'lat']] = df5['공간정보'].progress_apply(extract_center_latlon)

100%|██████████| 96173/96173 [00:44<00:00, 2159.13it/s]


In [9]:
df5

,공간정보,나이,성별,lon,lat
0,POINT (211863.24 369313.275),80세 ~ 84세,F,127.132969,36.822413
1,POINT (211863.24 369313.275),70세 ~ 74세,F,127.132969,36.822413
2,POINT (138316.292 362843.505),85세 ~ 89세,F,126.309166,36.762181
3,POINT (166462.996 378076.684),85세 ~ 89세,F,126.623718,36.900859
4,POINT (166462.996 378076.684),90세 ~ 94세,F,126.623718,36.900859
...,...,...,...,...,...
96168,POINT (202829.446 318741.55),70세 ~ 74세,M,127.031528,36.366759
96169,POINT (208213.031 320403.21),75세 ~ 79세,F,127.091533,36.381702
96170,POINT (208186.245 320397.197),80세 ~ 84세,M,127.091235,36.381648
96171,POINT (208186.245 320397.197),75세 ~ 79세,F,127.091235,36.381648


In [10]:
import requests

# Kakao REST API 키
KAKAO_API_KEY = 'KakaoAK 498d899fd084b23f98e07886d41630fc'

# 주소 캐시 딕셔너리
address_cache = {}

# 고유 좌표 추출
unique_coords = df5[['lon', 'lat']].drop_duplicates().dropna()

# Kakao 역지오코딩 함수
def kakao_reverse_geocode(lat, lon):
    try:
        url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        headers = {"Authorization": KAKAO_API_KEY}
        params = {"x": lon, "y": lat}
        response = requests.get(url, headers=headers, params=params, timeout=5)
        if response.status_code == 200:
            documents = response.json().get('documents')
            if documents:
                return documents[0]['address']['address_name']
    except:
        return None
    return None

# 고유 좌표에 대해 주소 조회
from tqdm import tqdm
for _, row in tqdm(unique_coords.iterrows(), total=len(unique_coords)):
    lat, lon = row['lat'], row['lon']
    key = (lat, lon)
    address_cache[key] = kakao_reverse_geocode(lat, lon)

# 주소 매핑
df5['실주소'] = df5.apply(lambda row: address_cache.get((row['lat'], row['lon'])), axis=1)

100%|██████████| 57790/57790 [6:25:17<00:00,  2.50it/s]


In [11]:
df5

,공간정보,나이,성별,lon,lat,실주소
0,POINT (211863.24 369313.275),80세 ~ 84세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
1,POINT (211863.24 369313.275),70세 ~ 74세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
2,POINT (138316.292 362843.505),85세 ~ 89세,F,126.309166,36.762181,충남 태안군 태안읍 상옥리 산 133-1
3,POINT (166462.996 378076.684),85세 ~ 89세,F,126.623718,36.900859,충남 당진시 채운동 1128
4,POINT (166462.996 378076.684),90세 ~ 94세,F,126.623718,36.900859,충남 당진시 채운동 1128
...,...,...,...,...,...,...
96168,POINT (202829.446 318741.55),70세 ~ 74세,M,127.031528,36.366759,충남 공주시 이인면 이곡리 191-2
96169,POINT (208213.031 320403.21),75세 ~ 79세,F,127.091533,36.381702,충남 공주시 이인면 목동리 335-1
96170,POINT (208186.245 320397.197),80세 ~ 84세,M,127.091235,36.381648,충남 공주시 이인면 목동리 335-1
96171,POINT (208186.245 320397.197),75세 ~ 79세,F,127.091235,36.381648,충남 공주시 이인면 목동리 335-1


In [12]:
df5.to_csv('노인인구_주소변환.csv', index=False, encoding='cp949')

#### 저장된 데이터 불러와서 가공 진행

In [18]:
df5 = pd.read_csv('/content/노인인구_주소변환.csv', encoding='cp949')

df5

,공간정보,나이,성별,lon,lat,실주소
0,POINT (211863.24 369313.275),80세 ~ 84세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
1,POINT (211863.24 369313.275),70세 ~ 74세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
2,POINT (138316.292 362843.505),85세 ~ 89세,F,126.309166,36.762181,충남 태안군 태안읍 상옥리 산 133-1
3,POINT (166462.996 378076.684),85세 ~ 89세,F,126.623718,36.900859,충남 당진시 채운동 1128
4,POINT (166462.996 378076.684),90세 ~ 94세,F,126.623718,36.900859,충남 당진시 채운동 1128
...,...,...,...,...,...,...
96168,POINT (202829.446 318741.55),70세 ~ 74세,M,127.031528,36.366759,충남 공주시 이인면 이곡리 191-2
96169,POINT (208213.031 320403.21),75세 ~ 79세,F,127.091533,36.381702,충남 공주시 이인면 목동리 335-1
96170,POINT (208186.245 320397.197),80세 ~ 84세,M,127.091235,36.381648,충남 공주시 이인면 목동리 335-1
96171,POINT (208186.245 320397.197),75세 ~ 79세,F,127.091235,36.381648,충남 공주시 이인면 목동리 335-1


In [19]:
df5 = df5.dropna(axis=0)

In [21]:
# 천인시만 필터링
df5 = df5[df5['실주소'].str.contains('천안시')]

df5

,공간정보,나이,성별,lon,lat,실주소
0,POINT (211863.24 369313.275),80세 ~ 84세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
1,POINT (211863.24 369313.275),70세 ~ 74세,F,127.132969,36.822413,충남 천안시 서북구 성정동 823
934,POINT (218404.523 363661.497),70세 ~ 74세,F,127.206150,36.771379,충남 천안시 동남구 목천읍 신계리 300-1
1512,POINT (214301.344 367512.304),80세 ~ 84세,F,127.160262,36.806150,충남 천안시 동남구 원성동 506-2
3655,POINT (218450.22 363698.151),70세 ~ 74세,F,127.206662,36.771709,충남 천안시 동남구 목천읍 신계리 291-10
...,...,...,...,...,...,...
96080,POINT (211553.128 368646.01),65세 ~ 69세,M,127.129483,36.816404,충남 천안시 동남구 봉명동 144-6
96081,POINT (211571.627 368911.526),65세 ~ 69세,M,127.129694,36.818796,충남 천안시 동남구 봉명동 161-24
96082,POINT (211509.61 368603.865),65세 ~ 69세,M,127.128994,36.816024,충남 천안시 동남구 봉명동 180
96083,POINT (211499.97 368725.694),60세 ~ 64세,M,127.128888,36.817122,충남 천안시 동남구 봉명동 165-1


In [22]:
df5['나이'].unique()

array(['80세 ~ 84세', '70세 ~ 74세', '75세 ~ 79세', '85세 ~ 89세', '90세 ~ 94세',
       '65세 ~ 69세', '95세 ~ 99세', '60세 ~ 64세', '100세 ~ 104세', '45세 ~ 49세',
       '55세 ~ 59세', '50세 ~ 54세', '105세 ~ 109세', '40세 ~ 44세', '0세 ~ 4세',
       '35세 ~ 39세'], dtype=object)

#### 인구수

In [32]:
import pandas as pd
from collections import defaultdict

# 1. 이름 → 인덱스 매핑
name_to_index = {name: idx for idx, name in enumerate(data['이름'])}

# 2. 이름별 나이대 개수 초기화
name_func_counts = defaultdict(lambda: defaultdict(int))

# 3. 주소 기준으로 legal_to_admin 통해 이름 매칭 후 카운팅
for i, row in df5.iterrows():
    address = row['실주소']
    age_group = row['나이']

    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_func_counts[admin][age_group] += 1
            break  # 중복 방지

# 4. 데이터프레임 복사
merged5 = data.copy()

# 5. 고유한 나이대 수집 및 정렬
all_age_groups = set()
for counts in name_func_counts.values():
    all_age_groups.update(counts.keys())

# 숫자로 정렬할 수 있게 앞 숫자 기준으로 소팅
sorted_age_groups = sorted(
    all_age_groups,
    key=lambda x: int(x.split('세')[0].split('~')[0].strip()) if '~' in x else 999
)

# 6. 각 나이대별 컬럼 추가
for age in sorted_age_groups:
    merged5[age] = merged5['이름'].apply(lambda x: name_func_counts[x][age])

# 결측값 채우고 정수형 변환
merged5.fillna(0, inplace=True)
for age in sorted_age_groups:
    merged5[age] = merged5[age].astype(int)

# 7. 컬럼 순서 정리: 기본 정보 + 정렬된 나이대 순서로 재정렬
basic_cols = ['구', '구분', '이름']
merged5 = merged5[basic_cols + sorted_age_groups]

In [33]:
merged5

,구,구분,이름,0세 ~ 4세,35세 ~ 39세,40세 ~ 44세,45세 ~ 49세,50세 ~ 54세,55세 ~ 59세,60세 ~ 64세,65세 ~ 69세,70세 ~ 74세,75세 ~ 79세,80세 ~ 84세,85세 ~ 89세,90세 ~ 94세,95세 ~ 99세,100세 ~ 104세,105세 ~ 109세
0,동남구,읍·면,목천읍,2,0,0,0,0,0,11,177,262,363,349,199,66,22,1,0
1,동남구,읍·면,풍세면,0,0,0,3,5,18,37,193,174,209,188,115,41,17,1,0
2,동남구,읍·면,광덕면,0,0,0,1,2,4,12,104,170,188,161,102,40,15,3,0
3,동남구,읍·면,북면,0,0,0,0,2,2,23,122,212,183,200,105,26,6,0,0
4,동남구,읍·면,성남면,0,0,0,1,2,12,127,152,173,178,156,106,27,9,1,0
5,동남구,읍·면,수신면,0,0,0,0,1,6,45,68,106,123,142,64,24,4,0,0
6,동남구,읍·면,병천면,0,0,0,0,2,4,29,164,205,204,198,120,41,16,3,0
7,동남구,읍·면,동면,0,0,0,0,0,2,21,68,132,168,141,99,27,18,1,0
8,동남구,행정동명,중앙동,0,0,0,0,1,5,9,25,61,81,65,32,11,2,0,0
9,동남구,행정동명,문성동,0,0,0,0,0,1,5,37,67,148,163,70,24,4,0,0


In [34]:
from collections import defaultdict

name_to_indices = defaultdict(set)  # 각 이름별 포함된 행 인덱스
unique_matched_rows = set()         # 전체에서 중복 제거된 행 인덱스

for i, row in df5.iterrows():
    address = row['실주소']
    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_to_indices[admin].add(i)
                    unique_matched_rows.add(i)  # 모든 이름에서 중복 없이 포함된 행 모음
            break  # 첫 번째 일치한 법정동만 사용하여 중복 방지

# 이름별 포함된 행 수 출력
total = 0
for name in data['이름']:
    count = len(name_to_indices[name])
    print(f"🔹 {name}: {count}개 행 포함")
    total += count

# 최종 통계
print(f"\n이름별 포함된 행 수 합계 (중복 포함): {total}")
print(f"중복 없이 포함된 전체 행 수: {len(unique_matched_rows)}")
print(f"df5 전체 행 수: {len(df5)}")

🔹 목천읍: 1452개 행 포함
🔹 풍세면: 1001개 행 포함
🔹 광덕면: 802개 행 포함
🔹 북면: 881개 행 포함
🔹 성남면: 944개 행 포함
🔹 수신면: 583개 행 포함
🔹 병천면: 986개 행 포함
🔹 동면: 677개 행 포함
🔹 중앙동: 292개 행 포함
🔹 문성동: 519개 행 포함
🔹 원성1동: 472개 행 포함
🔹 원성2동: 408개 행 포함
🔹 봉명동: 445개 행 포함
🔹 일봉동: 604개 행 포함
🔹 신방동: 863개 행 포함
🔹 청룡동: 967개 행 포함
🔹 신안동: 985개 행 포함
🔹 성환읍: 2747개 행 포함
🔹 성거읍: 1174개 행 포함
🔹 직산읍: 1704개 행 포함
🔹 입장면: 1333개 행 포함
🔹 성정1동: 838개 행 포함
🔹 성정2동: 722개 행 포함
🔹 쌍용1동: 1592개 행 포함
🔹 쌍용2동: 1592개 행 포함
🔹 쌍용3동: 1592개 행 포함
🔹 백석동: 498개 행 포함
🔹 불당1동: 462개 행 포함
🔹 불당2동: 462개 행 포함
🔹 부성1동: 278개 행 포함
🔹 부성2동: 895개 행 포함

이름별 포함된 행 수 합계 (중복 포함): 28770
중복 없이 포함된 전체 행 수: 23313
df5 전체 행 수: 23313


In [35]:
# 가공된 파일 csv 다운로드
from google.colab import files
merged5.to_csv('천안시_노인인구수.csv', index=False, encoding='utf-8-sig')
files.download('천안시_노인인구수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### 인구수(성별구분)

In [45]:
import pandas as pd
from collections import defaultdict

# 1. 이름 → 인덱스 매핑
name_to_index = {name: idx for idx, name in enumerate(data['이름'])}

# 2. 이름별 (성별, 나이대) 개수 초기화
name_func_counts = defaultdict(lambda: defaultdict(int))

# 3. 주소 기준으로 legal_to_admin 통해 이름 매칭 후 카운팅
for i, row in df5.iterrows():
    address = row['실주소']
    age_group = row['나이']
    gender = row['성별']

    key = f"{gender}({age_group})"

    for legal, admin_list in legal_to_admin.items():
        if legal in address:
            for admin in admin_list:
                if admin in name_to_index:
                    name_func_counts[admin][key] += 1
            break  # 중복 방지

# 4. 데이터프레임 복사
merged6 = data.copy()

# 5. 고유한 성별/나이대 수집 및 정렬
all_keys = set()
for counts in name_func_counts.values():
    all_keys.update(counts.keys())

# 정렬 함수 정의 (성별 + 나이대)
def sort_key(k):
    gender, age = k.split('(')
    age = age.strip('세 )~')  # 숫자만 남김
    try:
        return (0 if gender == '남자' else 1, int(age.split()[0]))
    except:
        return (2, 999)

sorted_keys = sorted(all_keys, key=sort_key)

# 6. 각 성별/나이대별 컬럼 추가
for key in sorted_keys:
    merged6[key] = merged6['이름'].apply(lambda x: name_func_counts[x][key])

# 결측값 채우고 정수형 변환
merged6.fillna(0, inplace=True)
for key in sorted_keys:
    merged6[key] = merged6[key].astype(int)

# 7. 컬럼 순서 정리: 기본 정보 + 정렬된 성별/나이대 순서
basic_cols = ['구', '구분', '이름']
merged6 = merged6[basic_cols + sorted_keys]

In [46]:
merged6

,구,구분,이름,M(55세 ~ 59세),M(100세 ~ 104세),M(85세 ~ 89세),M(80세 ~ 84세),F(70세 ~ 74세),M(45세 ~ 49세),M(70세 ~ 74세),...,M(35세 ~ 39세),M(60세 ~ 64세),F(80세 ~ 84세),F(105세 ~ 109세),F(60세 ~ 64세),F(100세 ~ 104세),M(95세 ~ 99세),M(0세 ~ 4세),M(75세 ~ 79세),F(65세 ~ 69세)
0,동남구,읍·면,목천읍,0,0,67,139,139,0,123,...,0,6,210,0,5,1,12,1,142,84
1,동남구,읍·면,풍세면,8,0,30,79,78,3,96,...,0,23,109,0,14,1,7,0,101,92
2,동남구,읍·면,광덕면,3,0,33,61,86,0,84,...,0,7,100,0,5,3,5,0,71,47
3,동남구,읍·면,북면,0,0,36,83,107,0,105,...,0,12,117,0,11,0,1,0,77,68
4,동남구,읍·면,성남면,8,0,35,58,84,1,89,...,0,81,98,0,46,1,2,0,81,86
5,동남구,읍·면,수신면,5,0,29,60,58,0,48,...,0,27,82,0,18,0,1,0,47,35
6,동남구,읍·면,병천면,3,0,43,88,113,0,92,...,0,11,110,0,18,3,3,0,86,73
7,동남구,읍·면,동면,1,0,33,54,68,0,64,...,0,8,87,0,13,1,4,0,70,36
8,동남구,행정동명,중앙동,2,0,9,21,43,0,18,...,0,1,44,0,8,0,0,0,35,18
9,동남구,행정동명,문성동,1,0,21,56,42,0,25,...,0,3,107,0,2,0,0,0,52,27


In [48]:
# 가공된 파일 csv 다운로드
from google.colab import files
merged6.to_csv('천안시_노인인구수_성별구분.csv', index=False, encoding='utf-8-sig')
files.download('천안시_노인인구수_성별구분.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**가공 과정**

1. polygon 정보 > 위경도로 변환
2. 위경도 좌표 > 카카오 API 사용하여 실주소로 변경 (6시간 걸림...)
3. 실주소 칼럼 생성 후 csv 저장
----
<저장된 csv 파일 불러와서>
4. NaN 존재하는 행 제거
5. 실주소에 '천안시' 해당되는 행만 추출
6. legal_to_admin 함수 이용하여 같은 행정동에 포함되는 행 찾은 후 각 행 값 합산하여 dataframe 생성
7. 행정구역 csv에 해당 dataframe 병합하여 새로운 csv 파일 생성

## 세부지표